In [1]:
import mlflow

mlflow.set_tracking_uri('http://ec2-18-141-141-49.ap-southeast-1.compute.amazonaws.com:5000')

In [2]:
mlflow.set_experiment('Exp 4 - Handling Imbalanced Data')

2025/08/03 15:10:29 INFO mlflow.tracking.fluent: Experiment with name 'Exp 4 - Handling Imbalanced Data' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-bucket-vietnq/522426905832806053', creation_time=1754208629645, experiment_id='522426905832806053', last_update_time=1754208629645, lifecycle_stage='active', name='Exp 4 - Handling Imbalanced Data', tags={}>

In [3]:
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [4]:
df = pd.read_csv('dataset.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [5]:
def run_imbalanced_experiments(imbalance_method):
    ngram_range = (1,3)
    max_features = 1000
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)

    if imbalance_method == "class_weights":
        class_weight = "balanced"
    else:
        class_weight = None
        if imbalance_method == "SMOTE":
            smote = SMOTE(random_state=42)
            X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)
        elif imbalance_method == "ADASYN":
            adasyn = ADASYN(random_state=42)
            X_train_vec, y_train = adasyn.fit_resample(X_train_vec, y_train)
        elif imbalance_method == "RandomUnderSampler":
            rus = RandomUnderSampler(random_state=42)
            X_train_vec, y_train = rus.fit_resample(X_train_vec, y_train)
        elif imbalance_method == "SMOTEENN":
            smote_enn = SMOTEENN(random_state=42)
            X_train_vec, y_train = smote_enn.fit_resample(X_train_vec, y_train)

    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"Imbalance_{imbalance_method}_RandomForest_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "imbalance_handling")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with TF-IDF Trigrams, imbalance handling method={imbalance_method}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("imbalance_method", imbalance_method)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42, class_weight=class_weight)
        model.fit(X_train_vec, y_train)

        # Step 6: Make predictions and log metrics
        y_pred = model.predict(X_test_vec)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, Imbalance={imbalance_method}")
        confusion_matrix_filename = f"confusion_matrix_{imbalance_method}.png"
        plt.savefig(confusion_matrix_filename)
        mlflow.log_artifact(confusion_matrix_filename)
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_trigrams_imbalance_{imbalance_method}")

imbalanced_methods = ['SMOTE', 'ADASYN', 'RandomUnderSampler', 'class_weights', 'SMOTEENN']

for method in imbalanced_methods:
    run_imbalanced_experiments(method)

2025/08/03 15:27:29 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/03 15:27:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_SMOTE_RandomForest_TFIDF_Trigrams at: http://ec2-18-141-141-49.ap-southeast-1.compute.amazonaws.com:5000/#/experiments/522426905832806053/runs/0aa48925b6da43ab88fa810430d236d2
🧪 View experiment at: http://ec2-18-141-141-49.ap-southeast-1.compute.amazonaws.com:5000/#/experiments/522426905832806053


2025/08/03 15:28:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/03 15:28:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_ADASYN_RandomForest_TFIDF_Trigrams at: http://ec2-18-141-141-49.ap-southeast-1.compute.amazonaws.com:5000/#/experiments/522426905832806053/runs/807f5fc2411048999bc6149d6b1b0a67
🧪 View experiment at: http://ec2-18-141-141-49.ap-southeast-1.compute.amazonaws.com:5000/#/experiments/522426905832806053


2025/08/03 15:28:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/03 15:28:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_RandomUnderSampler_RandomForest_TFIDF_Trigrams at: http://ec2-18-141-141-49.ap-southeast-1.compute.amazonaws.com:5000/#/experiments/522426905832806053/runs/9a6fa5db6fdf4e8ab9bfa14fb3f776a2
🧪 View experiment at: http://ec2-18-141-141-49.ap-southeast-1.compute.amazonaws.com:5000/#/experiments/522426905832806053


2025/08/03 15:29:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/03 15:29:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_class_weights_RandomForest_TFIDF_Trigrams at: http://ec2-18-141-141-49.ap-southeast-1.compute.amazonaws.com:5000/#/experiments/522426905832806053/runs/93d7efe0d5654c3aa3e056f8f5d5b7aa
🧪 View experiment at: http://ec2-18-141-141-49.ap-southeast-1.compute.amazonaws.com:5000/#/experiments/522426905832806053


2025/08/03 15:30:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/03 15:31:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_SMOTEENN_RandomForest_TFIDF_Trigrams at: http://ec2-18-141-141-49.ap-southeast-1.compute.amazonaws.com:5000/#/experiments/522426905832806053/runs/ff65225f0f704cd1900838e9862e94f5
🧪 View experiment at: http://ec2-18-141-141-49.ap-southeast-1.compute.amazonaws.com:5000/#/experiments/522426905832806053
